In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import glob
import os


os.getcwd()


'C:\\Users\\maclo\\OneDrive\\Desktop\\geospatial_dashboard\\notebooks'

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import glob
import os

data_dir = "../data"
output_dir = "../outputs"


files = glob.glob(os.path.join(data_dir, "yellow_tripdata_*.csv"))
print("Files found:", files)

for file in files:
    print(f"\nProcessing file: {os.path.basename(file)}")
    
    chunks = pd.read_csv(file, chunksize=100000, low_memory=False)

    clean_chunks = []

    for chunk in chunks:
       
        chunk = chunk.dropna(subset=['pickup_longitude', 'pickup_latitude',
                                     'dropoff_longitude', 'dropoff_latitude'])

       
        chunk = chunk[
            (chunk.pickup_longitude.between(-80, -70)) &
            (chunk.pickup_latitude.between(35, 45)) &
            (chunk.dropoff_longitude.between(-80, -70)) &
            (chunk.dropoff_latitude.between(35, 45))
        ]

        clean_chunks.append(chunk)

  
    cleaned_df = pd.concat(clean_chunks, ignore_index=True)
    print(f"Rows after cleaning: {len(cleaned_df)}")

   
    output_file = os.path.join(output_dir, f"cleaned_{os.path.basename(file)}")
    cleaned_df.to_csv(output_file, index=False)
    print(f"Saved cleaned file: {output_file}")


Files found: ['../data\\yellow_tripdata_2015-01.csv', '../data\\yellow_tripdata_2016-01.csv', '../data\\yellow_tripdata_2016-02.csv', '../data\\yellow_tripdata_2016-03.csv']

Processing file: yellow_tripdata_2015-01.csv
Rows after cleaning: 12484055
Saved cleaned file: ../outputs\cleaned_yellow_tripdata_2015-01.csv

Processing file: yellow_tripdata_2016-01.csv
Rows after cleaning: 10720690
Saved cleaned file: ../outputs\cleaned_yellow_tripdata_2016-01.csv

Processing file: yellow_tripdata_2016-02.csv
Rows after cleaning: 11188872
Saved cleaned file: ../outputs\cleaned_yellow_tripdata_2016-02.csv

Processing file: yellow_tripdata_2016-03.csv
Rows after cleaning: 12011828
Saved cleaned file: ../outputs\cleaned_yellow_tripdata_2016-03.csv


In [2]:
import pandas as pd


df_cleaned = pd.read_csv("../outputs/cleaned_yellow_tripdata_2015-01.csv", nrows=5)
print(df_cleaned.head())
print("Shape:", df_cleaned.shape)

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2015-01-15 19:05:39   2015-01-15 19:23:42                1   
1         1  2015-01-10 20:33:38   2015-01-10 20:53:28                1   
2         1  2015-01-10 20:33:38   2015-01-10 20:43:41                1   
3         1  2015-01-10 20:33:39   2015-01-10 20:35:31                1   
4         1  2015-01-10 20:33:39   2015-01-10 20:52:58                1   

   trip_distance  pickup_longitude  pickup_latitude  RateCodeID  \
0           1.59        -73.993896        40.750111           1   
1           3.30        -74.001648        40.724243           1   
2           1.80        -73.963341        40.802788           1   
3           0.50        -74.009087        40.713818           1   
4           3.00        -73.971176        40.762428           1   

  store_and_fwd_flag  dropoff_longitude  dropoff_latitude  payment_type  \
0                  N         -73.974785         40.750618             1

In [4]:
import pandas as pd
import glob

files = glob.glob("../outputs/cleaned_yellow_tripdata_*.csv")
output_path = "../outputs/merged_cleaned_taxi_data.csv"

first = True
for file in files:
    print(f"Merging {file} ...")
    for chunk in pd.read_csv(file, chunksize=200000):  # 200k rows at a time
        chunk.to_csv(output_path, mode='a', index=False, header=first)
        first = False

print("✅ Merging completed successfully (chunked mode).")


Merging ../outputs\cleaned_yellow_tripdata_2015-01.csv ...
Merging ../outputs\cleaned_yellow_tripdata_2016-01.csv ...
Merging ../outputs\cleaned_yellow_tripdata_2016-02.csv ...
Merging ../outputs\cleaned_yellow_tripdata_2016-03.csv ...
✅ Merging completed successfully (chunked mode).


In [5]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

df = pd.read_csv("../outputs/merged_cleaned_taxi_data.csv", nrows=1000000)

gdf = gpd.GeoDataFrame(
    df,
    geometry=[Point(xy) for xy in zip(df.pickup_longitude, df.pickup_latitude)],
    crs="EPSG:4326"
)

gdf.to_file("../outputs/merged_sample.geojson", driver="GeoJSON")
print("✅ GeoDataFrame sample created successfully.")


✅ GeoDataFrame sample created successfully.
